In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import numpy as np
import pandas as pd
from kite_trade import *
from enctoken import get_kite
import time

kite = get_kite()

timerframe_list = [
"day",
"minute",
 "3minute",
 "5minute",
 "10minute",
 "15minute",
 "30minute",
 "60minute",]

# instument in nse
inst = pd.DataFrame(kite.instruments("NSE"))
inst_filter = inst.query('(name != "")').copy()
inst_filter.rename(columns = {"tradingsymbol":'Symbol'},inplace = True)
# inst_filter.query("Symbol. 'NIFTY'")
inst_filter[inst_filter.Symbol.str.contains('NIFTY BANK')]


,instrument_token,exchange_token,Symbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
9,260105,1016,NIFTY BANK,NIFTY BANK,0.0,None,0.0,0.0,0,EQ,INDICES,NSE


In [26]:
inst = pd.DataFrame(kite.instruments("NFO"))

In [1]:
# strik_price_ce = 40000
# strik_price_pe = 40000
# symbol_ce = f"NFO:BANKNIFTY23302{strik_price_ce}CE"
# symbol_pe = f"NFO:BANKNIFTY23302{strik_price_pe}PE"
trade_list = []
while(True):
    inside = 0 
    buy_sell = 0
    sp = kite.ltp('NSE:NIFTY BANK')['NSE:NIFTY BANK']['last_price']
    sp = sp - sp %100
    strik_price_ce = int(sp)
    strik_price_pe = int(sp)
    symbol_ce = f"NFO:BANKNIFTY23302{strik_price_ce}CE"
    symbol_pe = f"NFO:BANKNIFTY23302{strik_price_pe}PE"
    
    if datetime.now().minute % 5 == 0:
        print(trade_list)
        print(datetime.now())
        bn_begin = kite.ltp('NSE:NIFTY BANK')['NSE:NIFTY BANK']['last_price']
        op_begin_ce = kite.ltp(symbol_ce)[symbol_ce]['last_price']
        op_begin_pe = kite.ltp(symbol_pe)[symbol_pe]['last_price']
        
        time.sleep(15)
        bn_after = kite.ltp('NSE:NIFTY BANK')['NSE:NIFTY BANK']['last_price']
        op_after_ce = kite.ltp(symbol_ce)[symbol_ce]['last_price']
        op_after_pe = kite.ltp(symbol_pe)[symbol_pe]['last_price']
        
        if bn_after > bn_begin:
            print("buy:",op_after_ce,bn_after)
            buy_sell = 1
        elif bn_after < bn_begin:
            print("sell:",op_after_pe,bn_after)
            buy_sell = -1
        while(inside == 0 ):
            op_current_ce = kite.ltp(symbol_ce)[symbol_ce]['last_price']
            op_current_pe = kite.ltp(symbol_pe)[symbol_pe]['last_price']
            bn_current = kite.ltp('NSE:NIFTY BANK')['NSE:NIFTY BANK']['last_price']
      
            
            if buy_sell == 1:  
                # print(op_current_ce,bn_current)     
                if op_current_ce >= ( op_after_ce + 20):
                    print("sell with profit:",bn_current,op_current_ce, op_current_ce - op_after_ce)
                    inside = 1
                    trade_list.append([bn_current,op_current_ce,op_current_ce - op_after_ce])
                elif op_current_ce <= (op_after_ce - 10):
                    print("sell with loss:",bn_current,op_current_ce, op_current_ce - op_after_ce)
                    inside = 1
                    trade_list.append([bn_current,op_current_ce,op_current_ce - op_after_ce])
            elif buy_sell == -1:
                    if op_current_pe >= ( op_after_pe + 20):
                        print("sell with profit:",bn_current,op_current_pe, op_current_pe - op_after_pe)
                        inside = 1
                        trade_list.append([bn_current,op_current_pe, op_current_pe - op_after_pe])
                    elif op_current_pe <= (op_after_pe - 10):
                        print("sell with loss:",bn_current,op_current_pe, op_current_pe - op_after_pe)
                        inside = 1
                        trade_list.append([bn_current,op_current_pe, op_current_pe - op_after_pe])
            time.sleep(1)
